# <a id='toc1_'></a>[Optimized Options Writing](#toc0_)
Evaluating the potential for Options Writing in Two Investment Funds through the Prism of Greeks Analysis

**Disclosure**: I utitlized **ChatGPT V4.0** for English writting refinments and **Github's Copilot** for function and class documentations defined in `src` directory. 

**Table of contents**<a id='toc0_'></a>    
- [Optimized Options Writing](#toc1_)    
  - [Theory](#toc1_1_)    
    - [Option Greeks](#toc1_1_1_)    
    - [Greek Strategies](#toc1_1_2_)    
  - [Implementation](#toc1_2_)    
    - [Data](#toc1_2_1_)    
  - [Execution](#toc1_3_)    
  - [Discussion](#toc1_4_)    
  - [Further Discussion](#toc1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Theory](#toc0_)
### <a id='toc1_1_1_'></a>[Option Greeks](#toc0_)

**Delta (Δ)**: Delta measures how much the price of an option is expected to change per $1 change in the price of the underlying security. Delta values range between 0 and 1 for call options and -1 to 0 for put options.

Formula for Delta of a call option:
$\Delta_C = N(d_1)$

Formula for Delta of a put option:
$\Delta_P = -N(-d_1)$

Here, $N()$ denotes the cumulative distribution function of the standard normal distribution, and $d_1$ is calculated as:
$d_1 = \frac{\ln(\frac{S}{K}) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$

where:
- $S$ is the spot price of the underlying asset
- $K$ is the strike price
- $r$ is the risk-free interest rate
- $σ$ is the volatility of the return on the underlying asset
- $T$ is the time to maturity

**Gamma (Γ)**: Gamma measures the rate of change in the delta with respect to changes in the underlying price. Gamma is highest for at-the-money options and is lower for both in-the-money and out-of-the-money options.

Formula for Gamma:
$\Gamma = \frac{N'(d_1)}{S\sigma\sqrt{T}}$

$N'()$ is the standard normal probability density function.


**Vega (v)**: Vega measures the rate of change in the price of an option for every 1% change in underlying volatility. Vega does not have any directional bias; it increases as volatility increases and decreases as volatility decreases, regardless of the direction of the underlying price move.

Formula for Vega:
$v = SN'(d_1)\sqrt{T}$

**Theta (Θ)**: Theta measures the rate of decline in the value of an option due to the passage of time. It's also referred to as the time decay of the option. Theta is typically negative for purchased options, and positive for sold options.

Formula for Theta of a call option:
$\Theta_C = -\frac{SN'(d_1)\sigma}{2\sqrt{T}} - rKe^{-rT}N(d_2)$

Formula for Theta of a put option:
$\Theta_P = -\frac{SN'(d_1)\sigma}{2\sqrt{T}} + rKe^{-rT}N(-d_2)$

$d_2$ is calculated as:
$d_2 = d_1 - \sigma\sqrt{T}$

**Rho (ρ)**: Rho measures the sensitivity of an option or options portfolio to the interest rate. For call options, Rho is usually positive, while for put options, it's negative.

Formula for Rho of a call option:
$\rho_C = KTe^{-rT}N(d_2)$

Formula for Rho of a put option:
$\rho_P = -KTe^{-rT}N(-d_2)$


By understanding and applying the Greeks, options traders can potentially create more effective and efficient trading strategies, manage risk more effectively, and improve the likelihood of achieving their investment objectives.




### <a id='toc1_1_2_'></a>[Greek Strategies](#toc0_)

1. **Delta-Neutral Strategy**: The goal of a delta-neutral strategy is to reduce or eliminate the impact of price movement in the underlying asset. For example, if an options portfolio has a positive delta, a trader could offset that by shorting the equivalent delta amount of the underlying asset. The net effect is that the overall delta of the portfolio is brought close to zero, making it delta-neutral.

2. **Gamma Scalping**: This strategy is used by options traders to offset the risk of price movement in the underlying asset, particularly for a delta-neutral portfolio. In a gamma scalping strategy, the trader adjusts the delta of their portfolio back to zero when the price of the underlying asset moves. This adjustment is typically done by buying or selling shares of the underlying asset.

3. **Vega Positioning**: Vega measures sensitivity to volatility. If a trader expects an increase in volatility, they could construct a strategy to have a positive vega, meaning it would benefit from an increase in volatility. For example, long straddles and strangles have positive vega and could benefit from an increase in volatility. Conversely, if a trader expects a decrease in volatility, they could construct a negative vega strategy, like short straddles or strangles.

4. **Theta Strategy**: Since theta represents time decay, an options trader looking to benefit from the passage of time might write options, resulting in a positive theta portfolio. Essentially, they would benefit as the options lose value over time due to theta decay. This might involve strategies like covered calls, naked puts, or credit spreads.

5. **Rho Strategy**: If a trader expects interest rates to rise, they could construct a positive rho strategy, which will benefit from an increase in interest rates. This could involve buying calls or selling puts. Conversely, if a trader expects a fall in interest rates, they might construct a negative rho strategy, which would benefit from a decrease in interest rates. This could involve buying puts or selling call options.

## <a id='toc1_2_'></a>[Implementation](#toc0_)

### <a id='toc1_2_1_'></a>[Data](#toc0_)

For this particular project, I have selected the **Aram Index Fund** managed by Mofid Securities. The choice was dictated by the fund's substantial holdings in major stocks, a large number of which possess corresponding underlying options, making them suitable for our analysis.

Each stock from this fund is matched with a **put** and a *call* option bearing identical maturity and strike prices to ensure uniformity in calculations. The chosen options all possess maturities that lie within the upcoming **2-4 mont**h time frame.

In the following dataframe, we present the stocks from the Aram Index Fund that currently have active options in the market. Each row represents a single stock, paired with its corresponding call and put option. These options will form the basis of our subsequent analysis.

In [8]:
from src.data import read_fund_portfolio_options
fund_portfolio_options1 = read_fund_portfolio_options(fund='aram')
fund_portfolio_options1

,stock_symbol,holding,option_symbol,maturity_date,strike,type
0,خودرو,38372289,ضخود6015,1402-06-08,3000,call
1,خودرو,38372289,طخود6015,1402-06-08,3000,put
2,وتجارت,27555013,ضجار6010,1402-06-14,2942,call
3,وتجارت,27555013,طجار6010,1402-06-14,2942,put
4,وبصادر,30935774,ضصاد7008,1402-07-12,2797,call
5,وبصادر,30935774,طصاد7008,1402-07-12,2797,put
6,وبملت,57146213,ضملت7003,1402-07-26,4370,call
7,وبملت,57146213,طملت7003,1402-07-26,4370,put
8,شپنا,32895084,ضشنا6009,1402-06-01,14000,call
9,شپنا,32895084,طشنا6009,1402-06-01,14000,put


## <a id='toc1_3_'></a>[Execution](#toc0_)

In this section, we transition from theory to practice. Leveraging our understanding of option Greeks, we will analyze the Aram Index Fund's holdings and explore the potential strategies that could be adopted by the fund. The core objective is to determine the maximum number of put and call options that the fund could write based on its current stock holdings.

Writing options is a strategy used by funds to generate additional income from the premium collected. However, it's important to understand that this strategy comes with certain risks and, therefore, requires meticulous calculation and management. This is where option Greeks come into play - they offer insights into risk factors that affect the price of an option.

The Delta, for instance, measures the rate of change in the option price for every $1 change in the underlying stock price. By maintaining a Delta-neutral position, the fund can attempt to hedge against price movements in the underlying stocks. Meanwhile, Theta, which represents time decay, could be used to the fund's advantage as the options they write lose value over time, potentially leading to profit if they can buy the option back at a lower price or let it expire worthless.

In general, the Greeks help in understanding the risks associated with writing options, but they don't directly determine the number of options that a fund can write. The fund's holdings and the contract size are the primary determinants of how many options can be written. However, the Greeks can inform the strategy behind which options to write. Here is how you might consider the other Greeks:

**Gamma**: A portfolio with a high absolute Gamma value will see its Delta change rapidly with movements in the underlying asset price, requiring more frequent rebalancing to maintain a Delta-neutral position. Therefore, a fund might choose to write options with a lower Gamma to reduce the frequency and cost of rebalancing.

**Theta**: As mentioned before, Theta represents the time decay of an option's price. A fund might choose to write options with a higher absolute Theta value, as these options will lose value more rapidly (all else being equal), potentially increasing the profitability of the writing strategy.

**Vega**: Vega measures sensitivity to volatility. If a fund expects volatility to decrease, they might choose to write options with a high Vega, as these options will lose value as volatility decreases.

**Rho**: If the fund expects interest rates to rise, they might prefer to write options with a high negative Rho, as these options will lose value as interest rates increase.


In the context of this project, we have elected to utilize a **Delta-Neutral Strategy** for determining the number of options that the Aram Index Fund can write. Our choice is underpinned by the principle that this approach provides an effective balance of risk by mitigating the price risk associated with the underlying asset.

**While the other Greek measures – Gamma, Theta, Vega, and Rho – offer valuable insights into factors such as sensitivity to changes in the price of the underlying asset, time decay, volatility, and interest rates, they do not directly dictate the number of options that the fund can write.**

For instance, while Theta could indicate potential profitability from an option writing strategy due to time decay, and Vega could provide insights about changes in option price due to volatility, these measures do not provide concrete limits on the number of options that can be written, unlike Delta in a Delta-Neutral strategy.

Moreover, Gamma, which impacts the rate of change in Delta and might necessitate frequent rebalancing of the portfolio, introduces additional considerations about transaction costs and operational efficiency.

The following calculations and analyses are aimed at providing a snapshot of potential strategic moves that the Aram Index Fund could implement.

For this project, a range of custom functions and classes have been developed and utilized to streamline the calculations and enhance the efficiency of the overall process. Here is an overview of these components:

| Component | Location | Description |
|-----------|----------|-------------|
| `fetch_stock_history` | `src.data` module | A function dedicated to fetching the historical price data of each option and the corresponding underlying stock. |
| `add_rff` | Described in Project1 | A function designed to calculate and append the risk-free rate to our data. |
| `add_std` | Described in Project1 | A function implemented to compute and add the standard deviation, which is a measure of the amount of variation or dispersion of a set of values, to our data. |
| `Strategy` | `src.option` | A class which encompasses the calculations related to our chosen Delta-neutral strategy. |


In [13]:
from src.data import fetch_stoch_history
from src.data import add_rf, add_std
from src.options import Strategy

In [17]:
# define greeks' columns
fund_portfolio_options1['delta'] = None
fund_portfolio_options1['gamma'] = None
fund_portfolio_options1['vega'] = None
fund_portfolio_options1['theta'] = None
fund_portfolio_options1['rho'] = None

# define strategy column
fund_portfolio_options1['delta_neutral'] = None

# we iterate over each put and call option
for option in fund_portfolio_options1.iterrows():

    # We need historical price of underlying stock to calculate sigma
    symbol = option[1]['stock_symbol']
    stock_price_df = fetch_stoch_history(symbol)
    stock_price_df = add_rf(stock_price_df)
    stock_price_df = add_std(stock_price_df)
    date, S0, rf, std = stock_price_df.iloc[-1]

    maturity_date = option[1]['maturity_date']
    strike = option[1]['strike']
    is_call = option[1]['type'] == 'call'

    # calculate greeks
    delta, gamma, vega, theta, rho = (
        Strategy.add_greeks(K=strike, mat_date=maturity_date, is_call=is_call, today=str(date), S0=S0, rf=rf, sigma=std)
    )
    fund_portfolio_options1.loc[option[0], 'delta'] = delta
    fund_portfolio_options1.loc[option[0], 'gamma'] = gamma
    fund_portfolio_options1.loc[option[0], 'vega'] = vega
    fund_portfolio_options1.loc[option[0], 'theta'] = theta
    fund_portfolio_options1.loc[option[0], 'rho'] = rho

    # calculate number of contracts to write based on delta_neutral strategy 
    fund_portfolio_options1.loc[option[0], 'delta_neutral'] = Strategy.delta_neutral_strategy(delta, option[1]['holding'])

## <a id='toc1_4_'></a>[Discussion](#toc0_)

In [16]:
fund_portfolio_options1

,stock_symbol,holding,option_symbol,maturity_date,strike,type,delta,gamma,vega,theta,rho,delta_neutral
0,خودرو,38372289,ضخود6015,1402-06-08,3000,call,0.759085,0.000644,386.924971,-1026.897091,313.519671,50550.689216
1,خودرو,38372289,طخود6015,1402-06-08,3000,put,-0.759085,0.000644,386.924971,-271.31237,313.519671,-50550.689216
2,وتجارت,27555013,ضجار6010,1402-06-14,2942,call,0.410798,0.000718,422.3755,-856.429548,159.540838,67076.860271
3,وتجارت,27555013,طجار6010,1402-06-14,2942,put,-0.410798,0.000718,422.3755,-148.269965,159.540838,-67076.860271
4,وبصادر,30935774,ضصاد7008,1402-07-12,2797,call,0.373567,0.000719,451.59081,-593.876266,192.278558,82811.843543
5,وبصادر,30935774,طصاد7008,1402-07-12,2797,put,-0.373567,0.000719,451.59081,66.076921,192.278558,-82811.843543
6,وبملت,57146213,ضملت7003,1402-07-26,4370,call,0.760461,0.000208,803.494549,-1474.216132,765.834113,75146.776726
7,وبملت,57146213,طملت7003,1402-07-26,4370,put,-0.760461,0.000208,803.494549,-453.000053,765.834113,-75146.776726
8,شپنا,32895084,ضشنا6009,1402-06-01,14000,call,0.002843,0.000007,28.166747,-49.31404,3.220224,11571194.200826
9,شپنا,32895084,طشنا6009,1402-06-01,14000,put,-0.002843,0.000007,28.166747,3494.523425,3.220224,-11571194.200826


As we delve into the results displayed in the ensuing DataFrame, there are a few key points to note.

**Relationship between Puts and Calls**: An important factor to consider in the context of our Delta-neutral strategy is the relationship between the number of put and call options the fund can write. In this strategy, the number of put options to write is effectively the negative of the number of call options. This relationship is directly derived from their respective Delta values, as a put option has a negative Delta, indicating that its price moves inversely to the price of the underlying asset. Consequently, to maintain a Delta-neutral position, the fund would need to write a negative quantity of put options for each call option, effectively meaning it should buy, rather than write, these options.

**Understanding the Delta-Neutral Column**: The values presented in the 'Delta_neutral' column represent the number of option contracts. It is crucial to understand that each option contract typically consists of 1000 underlying stocks. Therefore, the figures shown are not the absolute number of individual options, but rather the number of option contracts, each corresponding to 1000 stocks. For example, a Delta_neutral value of 5 indicates the fund can write options on 5000 stocks (5 contracts * 1000 stocks per contract).

## <a id='toc1_5_'></a>[Further Discussion](#toc0_)

While our project specifically employed a Delta-neutral strategy, it's worth considering that a multitude of other potential strategies could be implemented, particularly when incorporating additional Greeks into the mix. For instance, we could consider a strategy that evaluates Delta, Vega, and Theta exposures simultaneously and aims to minimize a linear combination of them.

Such a multi-Greek strategy would expand the risk parameters under consideration, providing a more nuanced approach to managing the fund's options portfolio. In this setup, the strategy would not only hedge against price movements in the underlying asset (Delta), but also against changes in implied volatility (Vega) and the passage of time (Theta).

Though we didn't implement a multi-Greek strategy in this project, the concept would look something like the following:

In [84]:
from scipy.optimize import minimize

def calculate_exposure(n_options, row):
    # Calculate the exposure to each greek for selling n_options options
    delta_exposure = abs(row['holding'] - n_options[0] * row['delta'])
    vega_exposure = abs(n_options[0] * row['vega'])
    theta_exposure = abs(n_options[0] * row['theta'])
    return delta_exposure + vega_exposure + theta_exposure

def calculate_options_to_sell(row):
    # Initial guess for the number of options to sell
    n_options_initial = row['delta_neutral']
    # Bounds to ensure we sell a non-negative number of options
    bounds = [(0, None)]
    result = minimize(calculate_exposure, n_options_initial, args=(row,), bounds=bounds)
    return result.x[0]